In [1]:
# -*- coding: utf-8 -*-
import requests
import time
import pandas as pd
class WeChatArticlesInfo(object):
    """
    获取微信公众号的推文信息
    """
    COLUMNS = ['aid', 'title', 'cover_url', 'abstract', 'url', 'time']
    INFO_KEY = "app_msg_list"
    COUNT_KEY = "app_msg_cnt"
    def __init__(self, cookie:str, token:str):
        """
        Parameter:
            cookie: str
            token: str
        """
        self.sess = requests.session()
        self.search_url = "https://mp.weixin.qq.com/cgi-bin/searchbiz"
        self.appmsg_url = "https://mp.weixin.qq.com/cgi-bin/appmsg"
        self.headers = {
            "Cookie": cookie,
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.62 Safari/537.36"
        }
        self.params = {
            "lang": "zh_CN",
            "f": "json",
            "token": token
        }
        self.fakeid = {}

    def get_basic_info(self, name:str, begin:int=0, count:int=5):
        """
        查找公众号基础信息，主要用于获取fakeid进行下一步的获取
        Parameter:
            name: str, 公众号名称，尽可能准确
            begin: int, 起始页数
            count: int, 数量，1-5

        Return:
            list
        """
        params = {
            "query": name,
            "count": str(count),
            "action": "search_biz",
            "ajax": "1",
            "begin": str(begin)
        }
        self.params.update(params)
        # 公众号名称一定要准确，否则排序上会出问题
        result = self.sess.get(self.search_url,
                                 headers=self.headers,
                                 params=self.params)
        return result.json()["list"]

    def __get_articles_info(self, name:str, begin:int, count:int=5):
        """
        获取公众号文章的信息，包含url(key: link)可用于后续爬取
        Parameters:
            name: str, 公众号名称，尽可能准确
            begin: int, 起始页数
            count: int, 数量，1-5
        Returns
            json
            important keys:
            "app_msg_list": 公众号文章信息
            "app_msg_cnt": 公众号文章总数
        """
        fakeid = self.fakeid.get(name, None)
        if fakeid is None:
            self.fakeid[name] = self.get_basic_info(name)[0]["fakeid"]
            fakeid = self.fakeid[name]
        params = {
            "fakeid": fakeid,
            "query": "",
            "begin": str(begin),
            "count": str(count),
            "type": "9",
            "action": "list_ex"
        }
        self.params.update(params)
        data = self.sess.get(self.appmsg_url, headers=self.headers, params=self.params)
        return data.json()
    
    def get_articles_info_step(self, name:str, begin:int=0, count:int=5):
        """
        查找并提取文章的信息
        Parameter:
            name: str, 公众号名称，尽可能准确
            begin: int, 起始页数
            count: int, 数量，1-5

        Return:
            list
        """
        data = self.__get_articles_info(name=name, begin=begin, count=count)[WeChatArticlesInfo.INFO_KEY]
        map_dict = {'aid': 'aid',
                    'title': 'title', 
                    'cover_url': 'cover',
                    'abstract': 'digest',
                    'url': 'link', 
                    'time': 'update_time'}
        data = [{key: x[map_dict[key]] for key in map_dict} for x in data]
        return data
    
    def get_articles_info(self, name:str, begin:int=0, count:int=15):
        """
        查找并提取文章的信息
        Parameter:
            name: str, 公众号名称，尽可能准确
            begin: int, 起始页数
            count: int, 数量
            endtimestamp: int，终止的timestamp

        Return:
            pd.DataFrame
        """
        nsteps = int(count / 5) + 1
        step = 0
        data = []
        while 5 * step < count:
            data.extend(self.get_articles_info_step(name, 5*step, 5))
            step = step + 1
            #time.sleep(0.01)
        return pd.DataFrame.from_records(data)

cookie = "appmsglist_action_3885044594=card; pgv_pvid=9077891912; pgv_pvi=2132619264; RK=yFyYv1ZTW1; ptcz=43a2e5542ef329277f931eca3fc8448e476d0396534226771eeb312b19b7d460; ua_id=6pa7LEid8uNGYVXmAAAAAHSKaOBDMqhtf9ENX0Ig2MI=; mm_lang=zh_CN; eas_sid=j135z8e1P57130610504j4c126; tvfe_boss_uuid=ba8bb21b5b9dd930; o_cookie=971813059; pac_uid=1_971813059; ied_qq=o0971813059; openid2ticket_ochOR50hKgGUzeW0YjvmEs1Q39p8=cpHBN07q/9nsGgs8gO5fOjT3p72yZNUcLXnjH0JhPeI=; iip=0; uin_cookie=o0971813059; openid2ticket_oG3em0cZw6NcqvCbQkhZ-oPMqXR8=IYJnlAgO8rEbqqW5ENs+aZ+y7J1DYGAnl1ben4vlfZc=; pgv_si=s9294711808; uuid=e61f7e83ae47cd9e0420494acbf9caec; rand_info=CAESIM12mkr54gTzxX9jy/V23oLPf29G8omPqb3qWXzXrluN; slave_bizuin=3885044594; data_bizuin=3885044594; bizuin=3885044594; data_ticket=Dtzdzwt43k70ixV2IuaT1a728QdUo/p9/zTUiUnTf+Gy6PtFjOFLNuAjKA1qMY1M; slave_sid=NmNzS0NjNUQyaFJvT0tKbWxWMkh0dkxsVUFsWlF5WVhiemR2aTRIM05pV1ZfcWl6SndGM2h6TEp3U0g3cWpfQ2JpcHVQT3gxNFVXZG9UTmVyUDg5ZW5fMjFEcHFsNllYcGZ3NDRYZ213dURJeGJTUVY3Z1lFdk5FN0p4cEJKZGRZMmFEbnNJUWVheVlxYnFP; slave_user=gh_3aea65724b37; xid="
token = "1744312947"

In [6]:
wechatarticle = WeChatArticlesInfo(cookie=cookie, token=token)

In [7]:
result = wechatarticle.get_articles_info("清华大学", begin=0, count=20)

In [8]:
result.head()

,aid,title,cover_url,abstract,url,time
0,2659222233_1,清华大学计算机系张悠慧团队首次提出“类脑计算完备性”,https://mmbiz.qlogo.cn/mmbiz_jpg/HhoEMZZMsiaSz...,,http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw...,1602804420
1,2659222230_1,两位清华人获深圳经济特区建立40周年创新创业人物和先进模范人物表彰 | 陈旭参加深圳经济特区...,https://mmbiz.qlogo.cn/mmbiz_jpg/HhoEMZZMsiaSz...,,http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw...,1602743207
2,2659222210_1,20所中美高校参加“中美大学校长云论坛” 共商建设更开放、更融合、更有韧性的大学,https://mmbiz.qlogo.cn/mmbiz_jpg/HhoEMZZMsiaT2...,,http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw...,1602691028
3,2659222163_1,融合式教学，一个都不能少！,https://mmbiz.qlogo.cn/mmbiz_jpg/HhoEMZZMsiaTi...,千里之外，天涯咫尺,http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw...,1602575215
4,2659222162_1,早知道,https://mmbiz.qlogo.cn/mmbiz_jpg/HhoEMZZMsiaRk...,,http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw...,1602460860


In [9]:
result.loc[0]['url']

'http://mp.weixin.qq.com/s?__biz=MzA4OTIyMzgxMw==&mid=2659222233&idx=1&sn=95daca442349c60391652f7c61f2f98a&chksm=8b6a0533bc1d8c25cfb7b8fbce48ee54c510d96f03c4c44b79d82d75b9b9ea5cb1dac8dff2a7#rd'

In [10]:
pb = []
for i in range(10):
    pb.extend(wechatarticle.get_basic_info("清华大学", i*5, (i+1)*5))

In [13]:
sorted(list(set([x['nickname'] for x in pb])))

['iCenter清华大学创客空间',
 '清华大学',
 '清华大学CIDEG',
 '清华大学MEM教育中心',
 '清华大学丘成桐数学科学中心',
 '清华大学中国农村研究院',
 '清华大学互联网产业研究院',
 '清华大学五道口金融学院',
 '清华大学体育代表队',
 '清华大学公共管理学院',
 '清华大学出版社',
 '清华大学出版社教学服务平台',
 '清华大学医学院',
 '清华大学医院',
 '清华大学国情研究院',
 '清华大学国际与地区研究院',
 '清华大学国际教育',
 '清华大学图书馆',
 '清华大学天津高端装备研究院',
 '清华大学学生会',
 '清华大学学生公益',
 '清华大学学生社团',
 '清华大学学生艺术团',
 '清华大学小研在线',
 '清华大学小研在身边',
 '清华大学工业工程系',
 '清华大学建筑学院',
 '清华大学建筑学院景观学系',
 '清华大学建筑节能研究中心',
 '清华大学心理学系',
 '清华大学教育基金会',
 '清华大学教育研究',
 '清华大学文体之声',
 '清华大学文化经济研究院',
 '清华大学新清华学堂',
 '清华大学新雅书院',
 '清华大学法学院',
 '清华大学深圳国际研究生院',
 '清华大学清新时报',
 '清华大学玉泉医院妇产科',
 '清华大学研究生教育',
 '清华大学社会实践',
 '清华大学积极心理学中心',
 '清华大学经济管理学院',
 '清华大学美术学院',
 '清华大学艺术博物馆',
 '清华大学苏世民书院',
 '清华大学藤影荷声',
 '清华大学计算机系校友会',
 '清华大学附属中学',
 '清华大学附属中学上地学校',
 '清华大学附属中学朝阳学校',
 '清华大学附属中学永丰学校',
 '清华大学附属北京清华长庚医院',
 '清华大学附属小学']